In [5]:
import pandas as pd
clean = "../../data/clean/"
with open("../../data/raw/LgaRankings_27_Offences.xlsx", "rb") as f:
    df_dict = pd.read_excel(f, sheet_name=None, encoding='latin-1', skiprows=0, header=[4,5], na_values=['nc'], skip_footer=7)


/Users/desultir/git/capstone/venv/lib/python3.6/site-packages/pandas/util/_decorators.py:177: FutureWarning: the 'skip_footer' keyword is deprecated, use 'skipfooter' instead
  return func(*args, **kwargs)


ValueError: Duplicated level name: "Local Government Area", assigned to level 1, is already used for level 0.

In [3]:
uns = pd.DataFrame(df_dict['Assault - domestic violence'].unstack())

uns.index = uns.index.rename(["Year","Type", "LGA"])
uns.columns = ['Assault - domestic violence']
uns

ValueError: Length of names must match number of levels in MultiIndex.

In [ ]:
cleaned_list = []

for key in df_dict:
    unstacked = pd.DataFrame(df_dict[key].unstack())

    unstacked.index = unstacked.index.rename(["Year","Type", "LGA"])
    unstacked.columns = [key]
    cleaned_list.append(unstacked)
    
final_df = cleaned_list[0]
for i in range(1,len(cleaned_list)):
    final_df = pd.merge(final_df, cleaned_list[i], how='inner', left_index=True, right_index=True)


In [ ]:
output_df = final_df.reset_index()
output_df = output_df[output_df["Type"] == 'Rate per 100,000 population']
output_df = output_df.dropna()
output_df.loc[output_df["LGA"] == "Greater Hume Shire", "LGA"] = "Greater Hume"
output_df = output_df.loc[output_df["LGA"] != "Lord Howe Island"]

In [ ]:
output_df.columns

In [ ]:
nonzeroes = output_df.astype(bool).sum(axis=0).sort_values(ascending=False)
nonzeroes = nonzeroes[nonzeroes== 602]
nonzeroes = nonzeroes.drop(['Type','LGA', 'Year'])
nonzeroes

In [ ]:
output_df.corr()["Other theft"].sort_values()

In [ ]:
output_df.to_csv("cleaned_target.csv")

In [10]:
output_df = pd.read_csv(clean+"cleaned_target.csv")

In [ ]:
import numpy as np
variance = output_df.var()
corr = output_df.loc[:,nonzeroes.index].corr()
while len(corr) > 5:
    maxvals = []
    argmaxs = []
    for i,x in corr.iterrows():
        argmax = 0
        maxval = 0

        for j, y in enumerate(x[1:]):
            if y > maxval and y != 1.0:
                maxval = y
                argmax = j
        #print(i, maxval, x.index[argmax+1])
        maxvals.append(maxval)
        argmaxs.append(argmax)

    #the most correlated - drop the one with less variance
    mostcorridx = np.argmax(maxvals)
    mostcorr = corr.index[mostcorridx]
    mostcorr_other = corr.index[argmaxs[mostcorridx]+1]
    if variance[mostcorr] > variance[mostcorr_other]:
        corr = corr.drop(mostcorr_other)
        corr = corr.drop(mostcorr_other, axis=1)
        print("Dropping "+mostcorr_other)
    else:
        corr = corr.drop(mostcorr)
        corr = corr.drop(mostcorr, axis=1)      
        print("Dropping "+mostcorr)

In [ ]:
variance.sort_values()

In [ ]:
corr = output_df.iloc[:,1:].corr()

In [ ]:
corr

In [11]:
# just keep chosen lgas and chosen targets
output_df = output_df[output_df["Year"] == 2016]
output_df = output_df[['LGA', 'Assault - domestic violence', 'Assault - non-domestic violence', 'Fraud', 'Sexual Offences', 'Malicious damage to property', 'Prohibited weapons offences',]]
output_df

,LGA,Assault - domestic violence,Assault - non-domestic violence,Fraud,Sexual Offences,Malicious damage to property,Prohibited weapons offences
362,Albury,544.4263,563.5963,525.2564,216.6203,1315.0580,316.3040
363,Armidale Regional,501.4681,653.2282,841.2787,221.0419,2217.0169,287.0245
364,Ballina,255.7125,321.4001,504.3870,154.8351,795.2893,152.4891
365,Bathurst Regional,438.7931,462.3841,686.4989,200.5237,1184.2695,226.4738
366,Bayside,315.9874,274.1388,728.4085,85.5167,584.6676,108.5638
367,Bega Valley,312.2606,341.7192,318.1524,153.1845,810.1102,153.1845
368,Bellingen,279.2213,263.7090,248.1967,116.3422,604.9794,93.0738
369,Berrigan,174.9883,139.9907,163.3224,81.6612,699.9533,279.9813
370,Blacktown,594.3767,453.1188,821.0799,163.9858,980.7504,148.7380
371,Bland,348.2010,315.0390,281.8770,215.5530,746.1449,381.3630


In [15]:
import numpy as np
output_df.iloc[:,1:] = np.log(output_df.iloc[:,1:])
output_df

,LGA,Assault - domestic violence,Assault - non-domestic violence,Fraud,Sexual Offences,Malicious damage to property,Prohibited weapons offences
362,Albury,6.299733,6.334338,6.263887,5.378146,7.181636,5.756704
363,Armidale Regional,6.217540,6.481927,6.734923,5.398352,7.703918,5.659568
364,Ballina,5.544054,5.772687,6.223344,5.042361,6.678706,5.027093
365,Bathurst Regional,6.084028,6.136396,6.531605,5.300932,7.076881,5.422629
366,Bayside,5.755702,5.613635,6.590862,4.448712,6.371043,4.687338
367,Bega Valley,5.743838,5.833989,5.762531,5.031643,6.697170,5.031643
368,Bellingen,5.632005,5.574846,5.514222,4.756536,6.405194,4.533393
369,Berrigan,5.164719,4.941576,5.095726,4.402579,6.551014,5.634723
370,Blacktown,6.387513,6.116154,6.710620,5.099780,6.888318,5.002186
371,Bland,5.852780,5.752696,5.641471,5.373207,6.614920,5.943752


In [ ]:
output_df.to_csv(clean+"filtered")